In [10]:
import google.generativeai as genai
import os
import json

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# prepare the large language model
GEMINI_API_KEY = os.environ.get("GOOGLE_CLOUD_API_KEY")

# Initialize the Gemini API
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(
    "gemini-1.5-pro-latest",
    generation_config=genai.GenerationConfig(
        max_output_tokens=8000,
        temperature=1,
    ),
)

In [8]:
# define some constants

question_count = 10

question_list_prompt = """

[
    {
        "team": "UCI Anteaters",
        "questions": [
            {
                "question": "What is the mascot of UC Irvine?",
                "answer_options": ["Anteater", "Aggie", "Triton", "Gaucho"]
                "correct_indices": [0]
            }
        ]
    }
]

The team should match the inputted team name. The answer_options represent possible answer choices a contestant can choose from. The correct_indices represent the index of the correct answer in the answer_options list.

"""

def generate_system_prompt(team1, context):

    return f"""
You are a sports trivia host, specializing in College Sports trivia.

You are known for creating engaging trivia questions, and you will be creating a list of questions.

You will create a list of {question_count} questions about this team:

{team1}

Here is an article detailing information about the team that you can use to create your questions:

{context}

Output your question list in a valid JSON format.

Here is an example output. If one of the teams was the UCI Anteaters, the output could look like this:

{question_list_prompt}
"""

In [11]:
def generate(prompt: str):
    response = model.generate_content(prompt)
    return response.text

# Generate the questions, and return as an array of questions
def generate_and_format_questions(team):
    # call milvus to query for the team's context

    # FIXME: For now, we will use a hardcoded context

    context = f"""{team} is a college sports team."""

    # generate the system prompt
    prompt = generate_system_prompt(team, context)
    # generate the questions
    print("Generating questions...")
    response = generate(prompt)
    # trim, format, and return the questions
    response = response.strip()
    response = response.replace("json", "")
    response = response.replace("`", "")
    print(response)
    # convert to JSON format
    response = json.loads(response)
    return response

def generate_embeddings_from_list(text_list: list):
    result = genai.embed_content(model="models/text-embedding-004", content=text_list, task_type="question_answering")
    for embedding in result['embedding']:
        print(str(embedding)[:50], '... TRIMMED]')
    return result

def generate_embeddings(text: str):
    result = genai.embed_content(model="models/text-embedding-004", content=text, task_type="retrieval_document")
    # Print just a part of the embedding to keep the output manageable
    print(str(result['embedding'])[:50], '... TRIMMED]')
    return result

def generate_and_save_questions(team):
    questions = generate_and_format_questions(team)

    # send question results to backend database
    # FIXME: For now, we will save the questions to a JSON file
    with open(f"{team}_questions.json", "w") as f:
        json.dump(questions, f)

In [12]:
generate_and_save_questions("UCI Anteaters")

Generating questions...
[
  {
    "team": "UCI Anteaters",
    "questions": [
      {
        "question": "What conference do the UCI Anteaters compete in?",
        "answer_options": ["Big West Conference", "Pac-12 Conference", "Mountain West Conference", "West Coast Conference"],
        "correct_indices": [0]
      },
      {
        "question": "In what year did UCI officially adopt the Anteater as its mascot?",
        "answer_options": ["1965", "1972", "1984", "1995"],
        "correct_indices": [0]
      },
      {
        "question": "Which UCI Anteaters team holds the record for most NCAA National Championships?",
        "answer_options": ["Men's Volleyball", "Men's Basketball", "Women's Soccer", "Baseball"],
        "correct_indices": [0]
      },
      {
        "question": "How many NCAA Men's Volleyball National Championships have the UCI Anteaters won?",
        "answer_options": ["2", "3", "4", "5"],
        "correct_indices": [3]
      },
      {
        "question": "W